In [33]:
import sqlite3
import plotly.graph_objects as go

In [34]:
# Connnect to `donations.sqlite`
conn = sqlite3.connect('donations.sqlite')
c = conn.cursor()

# Make SQLite smarter
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d
conn.row_factory = dict_factory
c = conn.cursor()

# Exploring the Dataset

In [40]:
# How big is our dataset
c.execute('''SELECT count(*) AS 'donation count' FROM donations''')
print(c.fetchone())

{'donation count': 4687884}


In [39]:
# Let's look at the data
c.execute('''SELECT * FROM donations LIMIT 10''')
data = c.fetchall()
for datum in data:
    print(datum)

{'donor_id': '00000ce845c00cbf0686c992fc369df4', 'created_at': '2013-12-17 21:47:14', 'amount': 50}
{'donor_id': '00002783bc5d108510f3f9666c8b1edd', 'created_at': '2016-02-02 18:34:27', 'amount': 99}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2016-10-25 20:15:11', 'amount': 10}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-01-16 01:11:20', 'amount': 15.51}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-01-16 14:20:10', 'amount': 100}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-01-16 14:26:19', 'amount': 9.69}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-01-16 15:46:57', 'amount': 13.75}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-02-01 18:53:25', 'amount': 150}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-02-03 14:20:57', 'amount': 10}
{'donor_id': '00002d44003ed46b066607c5455a999a', 'created_at': '2017-03-13 18:37:24', 'amo

In [ ]:
# Min, Max and Average Donation by year

In [58]:
# Number of Donors by month
c.execute("""SELECT count(DISTINCT donor_id) as 'Donor Count',
                    substr(created_at, 0, 8) as 'Month'
               FROM donations 
           GROUP BY substr(created_at, 0, 8);""")
donors_by_month = c.fetchall()

# move the data into two arrays to plot
donors = []
months = []
for donor in donors_by_month:
    donors.append(donor['Donor Count'])
    months.append(donor['Month'])

# Plot it
fig = go.Figure(
    data=[go.Bar(y=donors, x=months)],
    layout_title_text="New Donors by month"
)
fig.show()

In [64]:
buckets = ((0,9.99), (10.00, 49.99), (50.00, 99.99), (100.00, 499.99), (500.00, 999.99), (1000.00, 60001.00))

donor_buckets = []
donor_counts = []
for bucket in buckets:
    c.execute("SELECT count(donor_id) as 'Donor Count' FROM donations WHERE amount BETWEEN " + str(bucket[0]) + " AND " + str(bucket[1]))
    donor_by_month = c.fetchall()
    
    print(donor_by_month)
    donor_counts.append(donor_by_month[0]['Donor Count'])
    donor_buckets.append(str(bucket[0]) + " to " + str(bucket[1]))

fig = go.Figure(
    data=[go.Bar(x=donor_buckets, y=donor_counts)],
    layout_title_text="Number of Donations by bucket"
)
fig.show()

[{'Donor Count': 693178}]
[{'Donor Count': 2105385}]
[{'Donor Count': 1093310}]
[{'Donor Count': 739138}]
[{'Donor Count': 42932}]
[{'Donor Count': 13941}]


In [ ]:
#Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Bar(x= months, y= donors, name="New Donors"),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x= months, y= avdonation, name="Average Donations"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="Average Donations vs New Donors per Month"
)
# Set x-axis title
fig.update_xaxes(title_text="<b>Years</b>")
# Set y-axes titles
fig.update_yaxes(title_text="Total Donation <b>Amounts</b> ", secondary_y=False)
fig.update_yaxes(title_text="New <b>Donorts</b>", secondary_y=True)
fig.show()